# Day 5 - Supply Stacks
## Data Types

In [1]:
class Animal():
    def __init__(self, name, sound):
        self.name = name
        self.sound = sound
    
    def speak(self):
        print(f"{self.name}: {self.sound}")


class Cat(Animal):
    def __init__(self, name):
        super().__init__(name, 'Meeeeeowwwww')

        
class Dog(Animal):
    def __init__(self, name):
        super().__init__(name, 'Wooof!')

        
animals = [
    Cat('Oji'),
    Dog('Keli'),
    Cat('Skit Skat'),
    Dog('Tumba')
]

for animal in animals:
    animal.speak()


Oji: Meeeeeowwwww
Keli: Wooof!
Skit Skat: Meeeeeowwwww
Tumba: Wooof!


In [2]:
class Stack(object):
    def __init__(self, id, crates=[]):
        self.id = id
        self.crates = list(crates)

    def __repr__(self):
        crates =  "".join(f"[{crate}]" for crate in self.crates)
        return f"{self.id}: {crates}"
    
    def remove(self):
        """Removes the top crate on the stack and returns what it was."""
        return self.crates.pop()
    
    def add(self, crate):
        """Puts the crate at the top of the stack."""
        self.crates.append(crate)
    

def move(quantity, from_stack, to_stack, debug=False):
    for i in range(quantity):
        removed_crate = from_stack.remove()
        to_stack.add(removed_crate)
        if debug:
            print(f"Moved [{removed_crate}] from {from_stack.id} to {to_stack.id}")
    
    
class Step(object):
    def __init__(self, quantity, from_stack, to_stack):
        self.quantity = quantity
        self.from_stack = from_stack
        self.to_stack = to_stack
    
    def __repr__(self):
        return f"move {self.quantity} from {self.from_stack.id} to {self.to_stack.id}"
    
    def execute_one_at_a_time(self, debug=False):
        move(self.quantity, self.from_stack, self.to_stack, debug)
                
    def execute_all_at_once(self, debug=False):
        external_stack = Stack(-1)
        move(self.quantity, self.from_stack, external_stack, debug)
        move(self.quantity, external_stack, self.to_stack, debug)                
    
    def execute(self, debug=False, all_at_once=False):
        if all_at_once:
            self.execute_all_at_once(debug)
        else:
            self.execute_one_at_a_time(debug)

    
class Procedure(object):
    def __init__(self, stacks, steps):
        self.stacks = stacks
        self.steps = steps
    
    def __repr__(self):
        stacks = "\n".join(str(stack) for stack in self.stacks)
        steps = "\n".join(str(step) for step in self.steps)
        
        return f"{stacks}\n\n{steps}\n\n"
    
    def execute(self, debug=False, all_at_once=False):
        while self.steps:
            self.steps[0].execute(debug, all_at_once)
            self.steps.pop(0)
            if debug:
                print("------------------------")
                print(self)
        

def create_example_procedure():
    example_stacks = [
        Stack(1, 'ZN'),
        Stack(2, 'MCD'),
        Stack(3, 'P')
    ]

    example_steps = [
        Step(1, example_stacks[1], example_stacks[0]),
        Step(3, example_stacks[0], example_stacks[2]),
        Step(2, example_stacks[1], example_stacks[0]),
        Step(1, example_stacks[0], example_stacks[1])
    ]

    return Procedure(example_stacks, example_steps)

example_procedure = create_example_procedure()
print(example_procedure)
example_procedure.execute(debug=True)

1: [Z][N]
2: [M][C][D]
3: [P]

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2


Moved [D] from 2 to 1
------------------------
1: [Z][N][D]
2: [M][C]
3: [P]

move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2


Moved [D] from 1 to 3
Moved [N] from 1 to 3
Moved [Z] from 1 to 3
------------------------
1: 
2: [M][C]
3: [P][D][N][Z]

move 2 from 2 to 1
move 1 from 1 to 2


Moved [C] from 2 to 1
Moved [M] from 2 to 1
------------------------
1: [C][M]
2: 
3: [P][D][N][Z]

move 1 from 1 to 2


Moved [M] from 1 to 2
------------------------
1: [C]
2: [M]
3: [P][D][N][Z]






## Parsing

In [3]:
def find_blank_line(lines):
    """Return the index into the lines array of the first blank line."""
    for i in range(len(lines)):
        if len(lines[i]) == 0:
            return i
        
assert find_blank_line(["foo", "bar", "", "baz"]) == 2
assert find_blank_line(["", "", ""]) == 0
assert find_blank_line(["foo", "bar"]) is None


def get_crate(line, i):
    """Returns the crate at the ith position in the given line."""
    crate = line[1::4][i]
    if crate == ' ':
        return None
    else:
        return crate
    
assert get_crate('[C] [G] [Q] [L] [N] [D] [M] [D] [Q]', 0) == 'C'
assert get_crate('[C] [G] [Q] [L] [N] [D] [M] [D] [Q]', 4) == 'N'
assert get_crate('            [M] [S] [S]            ', 4) == 'S'
assert get_crate('            [M] [S] [S]            ', 0) is None


def get_crates(crate_lines, i):
    """Parse the ith stack and return it given the list of lines."""
    crates = ""
    
    for line in reversed(crate_lines):
        crate = get_crate(line, i)
        if crate is None:
            break
            
        crates += crate
        
    return crates
    
crate_lines = [
    '    [D]    ',
    '[N] [C]    ',
    '[Z] [M] [P]'
]
assert get_crates(crate_lines, 0) == 'ZN'


def parse_stacks(lines):
    last_line = lines[-1].strip()
    num_stacks = int(last_line.split(' ')[-1])
    
    crate_lines = lines[:-1]
    
    for i in range(num_stacks):
        yield Stack(i+1, get_crates(crate_lines, i))        
    
    
def parse_steps(lines, stacks):    
    for line in lines:
        step = line.split(" ")
        quantity = int(step[1])
        from_stack = int(step[3])
        to_stack = int(step[-1])
        yield Step(quantity, stacks[from_stack-1], stacks[to_stack-1])

        
def parse(input):
    lines = input.split('\n')
    
    blank_line = find_blank_line(lines)
    
    stacks = list(parse_stacks(lines[:blank_line]))
    steps = list(parse_steps(lines[blank_line+1:], stacks))

    return Procedure(stacks, steps)


example_input = """    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2"""

print(parse(example_input))

1: [Z][N]
2: [M][C][D]
3: [P]

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2




In [4]:
from aocd import get_data

raw_data = get_data(year=2022, day=5)
print(raw_data)

            [M] [S] [S]            
        [M] [N] [L] [T] [Q]        
[G]     [P] [C] [F] [G] [T]        
[B]     [J] [D] [P] [V] [F] [F]    
[D]     [D] [G] [C] [Z] [H] [B] [G]
[C] [G] [Q] [L] [N] [D] [M] [D] [Q]
[P] [V] [S] [S] [B] [B] [Z] [M] [C]
[R] [H] [N] [P] [J] [Q] [B] [C] [F]
 1   2   3   4   5   6   7   8   9 

move 1 from 7 to 4
move 3 from 4 to 7
move 4 from 3 to 4
move 5 from 6 to 9
move 1 from 8 to 1
move 2 from 3 to 2
move 3 from 4 to 6
move 1 from 3 to 6
move 9 from 7 to 1
move 1 from 2 to 4
move 3 from 4 to 9
move 4 from 9 to 8
move 6 from 8 to 2
move 1 from 8 to 6
move 1 from 4 to 1
move 11 from 1 to 7
move 1 from 4 to 7
move 7 from 2 to 5
move 5 from 6 to 3
move 2 from 4 to 3
move 2 from 5 to 9
move 1 from 8 to 6
move 3 from 1 to 5
move 2 from 6 to 9
move 1 from 4 to 8
move 2 from 2 to 1
move 7 from 5 to 9
move 6 from 3 to 6
move 1 from 2 to 5
move 1 from 3 to 8
move 12 from 7 to 3
move 1 from 1 to 8
move 2 from 1 to 9
move 20 from 9 to 5
move 1 from 1 to 7
move 5 

In [5]:
real_procedure = parse(raw_data)
real_procedure

1: [R][P][C][D][B][G]
2: [H][V][G]
3: [N][S][Q][D][J][P][M]
4: [P][S][L][G][D][C][N][M]
5: [J][B][N][C][P][F][L][S]
6: [Q][B][D][Z][V][G][T][S]
7: [B][Z][M][H][F][T][Q]
8: [C][M][D][B][F]
9: [F][C][Q][G]

move 1 from 7 to 4
move 3 from 4 to 7
move 4 from 3 to 4
move 5 from 6 to 9
move 1 from 8 to 1
move 2 from 3 to 2
move 3 from 4 to 6
move 1 from 3 to 6
move 9 from 7 to 1
move 1 from 2 to 4
move 3 from 4 to 9
move 4 from 9 to 8
move 6 from 8 to 2
move 1 from 8 to 6
move 1 from 4 to 1
move 11 from 1 to 7
move 1 from 4 to 7
move 7 from 2 to 5
move 5 from 6 to 3
move 2 from 4 to 3
move 2 from 5 to 9
move 1 from 8 to 6
move 3 from 1 to 5
move 2 from 6 to 9
move 1 from 4 to 8
move 2 from 2 to 1
move 7 from 5 to 9
move 6 from 3 to 6
move 1 from 2 to 5
move 1 from 3 to 8
move 12 from 7 to 3
move 1 from 1 to 8
move 2 from 1 to 9
move 20 from 9 to 5
move 1 from 1 to 7
move 5 from 5 to 3
move 1 from 8 to 7
move 2 from 8 to 3
move 2 from 6 to 5
move 1 from 6 to 4
move 18 from 3 to 2
move 1 from 

## Part 1

In [6]:
real_procedure.execute()
real_procedure

1: [P][S][C][D][F][Q][P][B][C][G][V][Q][Z][B][P][P][R][M][M][C][D][T][G][D][B][B][B][Z][N][C][T]
2: [L]
3: [N]
4: [S][D][G][Q][J][J][S][D][F][C][M][V][H][H][Q][L][G]
5: [S][F]
6: [G]
7: [M]
8: [F]
9: [N]




## Part 2

In [7]:
example_procedure = create_example_procedure()
example_procedure.execute(debug=True, all_at_once=True)

Moved [D] from 2 to -1
Moved [D] from -1 to 1
------------------------
1: [Z][N][D]
2: [M][C]
3: [P]

move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2


Moved [D] from 1 to -1
Moved [N] from 1 to -1
Moved [Z] from 1 to -1
Moved [Z] from -1 to 3
Moved [N] from -1 to 3
Moved [D] from -1 to 3
------------------------
1: 
2: [M][C]
3: [P][Z][N][D]

move 2 from 2 to 1
move 1 from 1 to 2


Moved [C] from 2 to -1
Moved [M] from 2 to -1
Moved [M] from -1 to 1
Moved [C] from -1 to 1
------------------------
1: [M][C]
2: 
3: [P][Z][N][D]

move 1 from 1 to 2


Moved [C] from 1 to -1
Moved [C] from -1 to 2
------------------------
1: [M]
2: [C]
3: [P][Z][N][D]






In [8]:
real_procedure = parse(get_data(year=2022, day = 5))
real_procedure.execute(all_at_once=True)
real_procedure

1: [L][B][P][F][J][C][Z][S][Q][D][R][G][D][P][B][F][G][C][M][N][H][P][Q][S][M][D][N][G][S][S][F]
2: [G]
3: [L]
4: [G][H][F][Z][V][V][C][M][T][N][D][B][P][C][Q][T][Q]
5: [B][J]
6: [C]
7: [M]
8: [B]
9: [D]


